# Classification /extraction 
--- 

we should consider only some properties derivated  from items but in this case we can build a vocabulary of property initially for two categories **Culture Representative** and **Culture Agnostic**. 


### How should we chose properties from item of Culture Agnostic and Culture Representative? 
### Training  Phase:

--- 

### Step 1: Building 1 vocabulary of properties associated to Culture Representative and Culture Agnostic 
In this phase, for every sample $ item_{i} $ associated to, we put all its properties  into a set  $S_{P} $
### Step 2: Embedded vectors of properties for every item 
In this case we associate a binary vector for every $item_{i} $ where for every possible property we associate 1 if it has that property $P_k$ otherwise 0.
### Step 3: Compute the centroids for every category C.A C.R, C.E.X
Now, after the preprocessed data we compute the centroid (like mean over every property $P_k$ on all samples) $C_{CA} $ and $C_{CR} $.
### Step 4: We compute the euclidean distance among properties and centroids
for every sample we compute distance among each property and centroids, so we will interpret strong relationship of some  properties wrt culture concepts while for concept more neutrals
### Step 5: Corresponding the similarity to every property with some methods wrt centroids
Every property, will have a weight computed following the importance of distance wrt centroids $C_{C.A} $ and  $ C_{C.R}$ : 
#### Case 1: Kernel Funtion (Kernel Density Estimation)
we will use kernel to give a more flexible weight : $ w_{gauss}(item_{i})=\exp{(-\frac{d_{C}(item_{i})}{2*\sigma^2})} $ .
We can compute **$\sigma$** like a constant such that influences the area of neighbours entities: 
we can compute it as: $\sigma=\frac{1}{\sqrt{2}*mean \space of \space distance}$, can be optimized empirically 
**The Gaussian kernel has the advantage of providing a gradual decrease in weight rather than a linear or inversely proportional decrease, allowing us to assign greater weight to nearby entities without excluding those that are further away.** .
We normalize all weights wrt the sum following weights wrt centroids

### Test Phase:

### Step 1: Compute distance among every element wrt to the both centroids
we compute per every $item_i$  and every feature of kind: $P_{123},P_{2345} $ ecc... the euclidean distance
### Step 2: Compute similarity for every test sample(with kernel approach) for both centroids
this pass helps us to understand in which direction a sample should go to the centroids for both centroids 
### Step 3: Given averaged sum of the importance of feature of samples 
in this step we compute, Culture and Agnostic_score= $ \sum_{fi=1}^{N_feat} item_{fi}* importance \space of  \space features $  the secnond term is measured in previous case
### Step 4 : Then, to emphasize the influence of entities closer to the centroids, we multiply the weighted score by the similarity.
So, ultimately, we multiply the weighted score by similarity to get a total score that is more sensitive to the entity's proximity to the centroids. This helps make a more accurate prediction based on how much the entity resembles the cultural or agnostic centers.

total_culture_score=$ Culture \space and  \space Agnostic_score * similarity \space of \space distance \space of \space test_sample $ repeated for every class score
total_agnostic_score = ... 
### Step 5: Classification based on best result
if total_culture_score > total agnostic_score  -> **Culture Representative**









# Step 0: Building a Vocabulary with all properties of training items

In [1]:
from datasets import load_dataset
import pandas as pd
import torch as t 
import numpy as np
from wikidata.client import Client
from itertools import islice
from collections import Counter
from concurrent.futures import ThreadPoolExecutor
from scipy.spatial.distance import euclidean

In [2]:
ds=load_dataset("sapienzanlp/nlp2025_hw1_cultural_dataset")
train_set=train_data=pd.DataFrame(ds["train"])
list_category=set(list(train_set.category))
print(list_category)
list_subcategory=set(list(train_set.category))
X_train=train_set.values

print(X_train) # stamp all dataset
#print(X_train.shape) 

{'films', 'sports', 'comics and anime', 'politics', 'history', 'geography', 'performing arts', 'transportation', 'visual arts', 'fashion', 'media', 'books', 'food', 'biology', 'gestures and habits', 'literature', 'music', 'architecture', 'philosophy and religion'}
[['http://www.wikidata.org/entity/Q32786' '916' '2012 film by M. Mohanan'
  ... 'films' 'film' 'cultural exclusive']
 ['http://www.wikidata.org/entity/Q371' '!!!'
  'American dance-punk band from California' ... 'music' 'musical group'
  'cultural representative']
 ['http://www.wikidata.org/entity/Q3729947' '¡Soborno!'
  'Mort & Phil comic' ... 'comics and anime' 'comics'
  'cultural representative']
 ...
 ['http://www.wikidata.org/entity/Q10779' 'Zwenkau'
  'city in the district of Leipzig, in the Free State of Saxony, Germany'
  ... 'geography' 'city' 'cultural exclusive']
 ['http://www.wikidata.org/entity/Q245296' 'zydeco'
  'music genre evolved in southwest Louisiana which blends blues, rhythm and blues, and music indigen

In [ ]:
def extract_entity_id(url):
    return url.strip().split("/")[-1]
#print(X_train.shape[0])
def extract_sample_from_cat(X,cat):
    l=list()
    for elem in X:
        if elem[4]==cat:
            l.append(elem[0])
    return np.array(l)
#entity_train=np.zeros(shape=
#def count_frequency_prop_id(list_sampl):
    

def dynamic_threshold(n):
    if n>=430:
        return 0.35
    elif n>=340:
        return 0.4
    elif n>=250:
        return 0.45
    else:
        return 0.50
# Implementation vocabulary on categories importance 
client = Client()
vocabulary=list()
def build_vocabulary(cat):
    vocabulary_subset=list()
    list_sample_cat=extract_sample_from_cat(X_train,cat)
    number_list_sample_cat=len(list_sample_cat)
    #print("sample category: "+cat)
    #print(len(list_sample_cat))
    set_properties=list()
    #set_properties=np.array(set_properties)
    for url in list_sample_cat:
        entity_train=extract_entity_id(url)
        item = client.get(entity_train, load=True)
        claim_item_i=item.data.get("claims",{})
       # print(claim_item_i)
        #print(len(claim_item_i))
        set_property_item=set()
        for prop_id,values in islice(claim_item_i.items(),len(claim_item_i)):
            #prop_entity = client.get(prop_id, load=True)
            #label = prop_entity.label
            #print(f"{prop_id} = {label} ({len(values)} statement{'s' if len(values) != 1 else ''})")
            set_property_item.add(str(prop_id))
        set_properties.append(set_property_item)

    print("Category:"+cat+"is collected")
    counter=Counter()
    for s in set_properties:
        for prop in s:
            counter[prop]+=1
    frequency_prop=counter
    #frequency_prop=count_frequency_prop_id(set_properties)
    sorted_prop=frequency_prop.most_common()
   # print(sorted_prop)
    #threshold=int(len(sorted_prop)*0.6)
    # ERROR
    #------
    th=0
    if(number_list_sample_cat>=429):
        th=0.45
    elif number_list_sample_cat>=300:
        th=0.5
    elif number_list_sample_cat>=150:
        th=0.6
    #th=dynamic_threshold(number_list_sample_cat)
    for prop_i,count in sorted_prop:
        support_categories=count/number_list_sample_cat
        if support_categories>=th: #min support 
            vocabulary_subset.append(prop_i)
        #if len(vocabulary_subset)>=6: #the first top k
            #break
    #print("Updated Vocabulary",vocabulary_subset) 
    #-----       
    return vocabulary_subset
        #set_property_item.clear()
    #print("intersection of sample belongs"+cat,set.intersection(*set_properties))

        #label = prop_entity.labels
        #print(claim_item_i)
with  ThreadPoolExecutor(max_workers=9) as executor:
    results=list(executor.map(build_vocabulary,list_category))

for vocab in results:
    vocabulary.extend(vocab)
vocabulary=list(set(vocabulary))
print("Updated Vocabulary: for categories",vocabulary)
#Updated Vocabulary: for categories ['P495', 'P279', 'P345', 'P910', 'P17', 'P571', 'P373', 'P18', 'P625', 'P856', 'P244', 'P646', 'P31', 'P131', 'P641']

Category:transportationis collected
Category:comics and animeis collected
Category:performing artsis collected
Category:historyis collected
Category:filmsis collected
Category:visual artsis collected
Category:geographyis collected
Category:sportsis collected
Category:booksis collected
Category:politicsis collected
Category:fashionis collected
Category:mediais collected
Category:gestures and habitsis collected
Category:foodis collected
Category:biologyis collected
Category:musicis collected
Category:literatureis collected
Category:architectureis collected
Category:philosophy and religionis collected
Updated Vocabulary: for categories ['P495', 'P279', 'P345', 'P910', 'P17', 'P571', 'P373', 'P18', 'P625', 'P856', 'P244', 'P646', 'P31', 'P131', 'P641']


In [ ]:
def extract_entity_id(url):
    return url.strip().split("/")[-1]
print(X_train.shape[0])
def extract_sample_from_cat(X,cat):
    l=list()
    for elem in X:
        if elem[4]==cat:
            l.append(elem[0])
    return np.array(l)
#entity_train=np.zeros(shape=
def count_frequency_prop_id(list_sampl):
    counter=Counter()
    for s in list_sampl:
        for prop in s:
            counter[prop]+=1
    return counter

# Implementation vocabulary on categories importance 
client = Client()
vocabulary=list()
def build_vocabulary(cat):
    vocabulary_subset=list()
    list_sample_cat=extract_sample_from_cat(X_train,cat)
    index = 1
    lunghezza = len(list_sample_cat)
    #print("sample category: "+cat)
    #print(len(list_sample_cat))
    set_properties=list()
    #set_properties=np.array(set_properties)
    for url in list_sample_cat:
        entity_train=extract_entity_id(url)
        item = client.get(entity_train, load=True)
        claim_item_i=item.data.get("claims",{})
       # print(claim_item_i)
        #print(len(claim_item_i))
        set_property_item=set()
        for prop_id,values in islice(claim_item_i.items(),len(claim_item_i)):
            #prop_entity = client.get(prop_id, load=True)
            #label = prop_entity.label
            #print(f"{prop_id} = {label} ({len(values)} statement{'s' if len(values) != 1 else ''})")
            set_property_item.add(str(prop_id))
        set_properties.append(set_property_item)
        #print(cat+"("+str(index)+"/"+str(lunghezza)+" samples):")
        index=index+1
        #print(set_properties)

    tutte_proprieta = [prop for gruppo in set_properties for prop in gruppo]

    # Conta le occorrenze
    conteggio = Counter(tutte_proprieta)

    # Ordina per frequenza decrescente e stampa
    for prop, freq in conteggio.most_common():
        print(f"{prop}: {freq}")

with  ThreadPoolExecutor(max_workers=10) as executor:
    executor.map(build_vocabulary,list_subcategory)
#
#for vocab in results:
#    vocabulary.extend(vocab)
#vocabulary=list(set(vocabulary))
#print("Updated Vocabulary: for categories",vocabulary)
    

# Step 1: Embedding con vettori binari 1/0 

In [9]:
# insert of binary vectors
def gaussian_kernel_estimation(X,d):
    sigma=1/(np.sqrt(2)*np.mean(d,axis=0))
    dist_sq=np.sum((X-d)**2)
    return np.exp(-dist_sq/(2*sigma**2))

# too slow :(
#----------
def embedding_sample(X,vocabulary):
    X_train_emb=np.zeros(shape=(X.shape[0],len(vocabulary)))
    print(f"Processing {X.shape[0]} sample")
    for j in range(0,X.shape[0]):
        ent=extract_entity_id(X[j][0])
        item_j = client.get(ent, load=True)
        claim_item_i=item_j.data.get("claims",{})
        set_p=set()
        for prop_id,__ in islice(claim_item_i.items(),len(claim_item_i)):
            
            set_p.add(str(prop_id))
        #print("processed:",claim_item_i)
        for v in range(0,len(vocabulary)):
            if vocabulary[v] in set_p:
                X_train_emb[j,v]=1
            else:
                X_train_emb[j,v]=0
        #np.arange(X_train_emb=X[j,6]
        print(f"Update X_train_emb {j}: {X_train_emb[j]}")
    return X_train_emb

#------------
#X_embed_train=embedding_sample(X_train,vocabulary)
culture_representative_train=embedding_sample(X_train[X_train[:,6]=='cultural representative'],vocabulary)
print("--Embedded Culture represetnative train--")
culture_agnostic_train=embedding_sample(X_train[X_train[:,6]=='cultural agnostic'],vocabulary)
print("--Embedded Cuture Agnostic--")
culture_exlusive_train=embedding_sample(X_train[X_train[:,6]=='cultural exclusive'],vocabulary)
print("--embedded Culture Exclusive--")

print(f"culture_agnostic_train: {culture_agnostic_train}")
print(f"cutlure agnostic shape {culture_agnostic_train.shape}")

print(f"culture_representative_train {culture_representative_train}")
print(f"culture_representative_train shape {culture_representative_train.shape}")

print(f"culture_exlusive_train {culture_exlusive_train}")
print(f"culture_exlusive_train shape {culture_exlusive_train.shape}")

centroid_agnostic=np.mean(culture_agnostic_train,axis=0)
centroid_representative=np.mean(culture_representative_train,axis=0)
centroid_exclusive=np.mean(culture_exlusive_train,axis=0)

weights_agnostic=np.zeros(shape=len(vocabulary))
weights_representative=np.zeros(shape=len(vocabulary))
weights_exclusive=np.zeros(shape=len(len(vocabulary)))
for i in range(len(vocabulary)):
    weights_agnostic[i]=gaussian_kernel_estimation(X_train[0:,i],centroid_agnostic)
    weights_representative[i]=gaussian_kernel_estimation(X_train[0:,i],centroid_representative)
    weights_exclusive[i]=gaussian_kernel_estimation(X_train[0:,i],centroid_exclusive)

# normalization
weights_agnostic[i]/=np.sum(weights_agnostic)
weights_representative[i]/=np.sum(weights_representative)
weights_exclusive[i]/=np.sum(weights_exclusive)

# I'm arrived here
#def predict_entity_score(x_sample,centroid_CA,centroid_CR,centroid_CE,weights_agnostic,weights_representative,weights_exclusive):





Processing 1688 sample
Update X_train_emb 0: [1. 0. 0. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 0.]
Update X_train_emb 1: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
Update X_train_emb 2: [1. 0. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 0.]
Update X_train_emb 3: [0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0.]
Update X_train_emb 4: [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0.]
Update X_train_emb 5: [0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 0.]
Update X_train_emb 6: [1. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 1. 0. 0.]
Update X_train_emb 7: [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
Update X_train_emb 8: [0. 0. 0. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0.]
Update X_train_emb 9: [0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0.]
Update X_train_emb 10: [0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 0. 0.]
Update X_train_emb 11: [0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0.]
Update X_train_emb 12: [0. 0. 1. 1. 0. 0. 1. 1. 0. 1. 1. 1. 1. 0. 0.]
Update X_train_emb 13: [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0.]
Update 

KeyboardInterrupt: 